# South Dunedin Sea Rise GW inundation workflow
0. Model updates
1. Setup one PEST interface
2. Setup another PEST interface
3. Prior Monte Carlo
4. Prior-data conflict resolution
5. Weight-balancing and objective function focusing
6. History matching
7. Monte Carlo projection scenario simulation
8. Some visualisation and interaction sprinkles

#### Make some imports

In [ ]:
%matplotlib widget
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from pathlib import Path

# from scipy import ndimage as nd

import flopy
print("flopy path: {}".format(flopy.__path__))
print("flopy version: {}".format(flopy.__version__))
import pyemu

#### also import some external scripts (where most of the work is done!)

In [ ]:
import sys
sys.path.append("..")
from scripts import utils
from scripts import build

#### clean up working dir

In [ ]:
_ = [shutil.rmtree(Path("..", d)) for d in ["Dunedin_Pred_base", 
                                            "Dunedin_SS_base",
                                            "template_hist_ss", 
                                            "template_proj_101",
                                            "master", 
                                            "master_hist_prior",
                                            "master_SSP3_10"] 
     if Path("..", d).exists()]

_____________
# Steady-State "history period" model build
The initial step in this notebook reads in the existing MODFLOW-2005 model built using groundwater vistas (Rekker, 2012), and re-builds a MODFLOW-NWT model using the script located `../scripts/build.py`. The 'build_from_orig' function in the build.py script performs this task

In [ ]:
base_m = build.build_from_orig(version='ss', run=True)

_______
# Transient projection model build
This step in the notebook reads in the existing MODFLOW-2005 model built using groundwater vistas (Rekker, 2012), and re-builds a transient MODFLOW-NWT model using the script located `../scripts/build.py`. The 'build_from_orig' function in the build.py script performs this task (`version='pred'`)

In [ ]:
pred_m = build.build_from_orig(version='pred', run=True)

## This is a "city" -- there are some city features

In [ ]:
ar = np.zeros((base_m.modelgrid.nrow, base_m.modelgrid.ncol))
drndf = pd.DataFrame(base_m.drn.stress_period_data[0])
ar[tuple(drndf[['i','j']].values.T)] = 1
fig, ax = plt.subplots(1, figsize=(8,6))
#cb = ax.imshow(ib1, alpha=0.75, cmap="Wistia", extent=m.sr.get_extent(),
 #              origin="upper", transform=mprj, zorder=11)
mapview = flopy.plot.PlotMapView(model=base_m, ax=ax)
c = mapview.plot_array(ar, cmap="jet", ax=ax)
mapview.plot_ibound()
plt.show()

## Quick plot of model and water level obs locs

In [ ]:
# ADD OBS/OUTPUTS
# prep actualy head observation data 
obs_locs = pd.read_csv(os.path.join("..", "obs_locs", "Cox_Operational_Targets.dat"),
                   index_col=0,delim_whitespace=True)
# loop over dataframe rows, get ij corresponding to well location
obs_locs[['i','j']] = obs_locs.apply(lambda x: 
                                     pd.Series(base_m.modelgrid.intersect(x.Easting, x.Northing)), 
                                     axis=1).astype(int)
obs_locs["k"] = 0 
obs_locs["sitename"] = obs_locs.index.map(lambda x: 
                                        x.replace('(','').replace(')','').replace('_','-'))
display(obs_locs)

plt.ion()
fig, ax = plt.subplots(1, figsize=(8,6))
top = np.ma.masked_array(base_m.dis.top.array, base_m.bas6.ibound.array[0]!=1)
ax = base_m.dis.top.plot(vmin=top.min(), vmax=top.max(), colorbar=True)
_ = ax.scatter(obs_locs.Easting, obs_locs.Northing, color='r', marker='v')

______________
# PEST interface setup -- around history model
This notebook makes extensive use of the `PstFrom` functionality in `pyemu` to set up multipliers on parameters (function defined in `build.py` module).

Observations are also defined, assigned initial values, and weights based on preliminary assumptions about error.


In [ ]:
nreals = 100 # number of realisations in ensemble
period = "hist"
version = "ss"
# pst, pf = build.setup_pst(base_m, nreals=nreals) # set-up PEST interface

### Let's do the first bit slowly:

#### 0. Start with a spatial reference and a variogram construction (used for geostats)

In [ ]:
sr = pyemu.helpers.SpatialReference(
    delr=base_m.modelgrid.delr, delc=base_m.modelgrid.delc, 
    xll=base_m.modelgrid.xoffset, yll=base_m.modelgrid.yoffset
)
# define a variagram and geostruct for drain conductance covariance 
v = pyemu.geostats.ExpVario(contribution=1.0, a=100)

#### 1. Instantiate PstFrom()

In [ ]:
t_d = f"template_{period}_{version}"
# Instantiate PstFrom object
pf = pyemu.utils.PstFrom(
    original_d=base_m.model_ws, 
    new_d=os.path.join("..", t_d), 
    spatial_reference=sr,
    start_datetime=base_m.start_datetime,
    zero_based=False, # the model is 1-based (MODFLOW start counting at 1) 
    remove_existing=True,
                )
# get active model domain (just cos it is handy to ave on a two letter variable)
ib = base_m.bas6.ibound.array

#### 2. Add an example set of parameters

In [ ]:
drn_gs = pyemu.geostats.GeoStruct(variograms=v, transform='log')
drn_files = [f for f in os.listdir(pf.new_d) if "DRN" in f and "dat" in f]

In [ ]:
pd.read_csv(Path(pf.new_d, drn_files[0]), header=None, delim_whitespace=True)

In [ ]:
# grid based drain conductance
pf.add_parameters(drn_files, 
                  par_type='grid',  # cell-by-cell 
                  lower_bound=0.1,  # par ranges define variance
                  upper_bound=10.,
                  index_cols=[0, 1, 2],  # tabular list-like input for drains so need these
                  use_cols=[4],  # column that relates to conductance
                  par_name_base='drncond-gr',
                  pargp='drncond-gr',
                  geostruct=drn_gs
                  )

#### 3. Add another set of parameters (on the same MODFLOW par... WAT?!)

In [ ]:
# global constant drain conductance
pf.add_parameters(drn_files, 
                  par_type='constant', # 1 additional global par
                  lower_bound=0.1, 
                  upper_bound=10.,
                  index_cols=[0,1,2],
                  use_cols=[4],
                  par_name_base='drncond-cn',
                  pargp='dcond-cn'
                 )

#### 4. Some pars that act on drain elevation

In [ ]:
v = pyemu.geostats.ExpVario(contribution=1.0,a=100)
drnelev_gs = pyemu.geostats.GeoStruct(variograms=v, transform='none')
_=pf.add_parameters(drn_files, 
              par_type='grid', #
              par_style='add',
              transform='none',
              lower_bound=-0.5, 
              upper_bound=0.5,
              index_cols=[0,1,2],
              use_cols=[3],
              par_name_base='drnelev-gr',
              pargp='delev-gr',
              geostruct=drnelev_gs
             )
# constant based drain elevation
_=pf.add_parameters(drn_files, 
                  par_type='constant', # 1 additional global par
                  par_style='add',
                  transform='none',
                  lower_bound=-0.5, 
                  upper_bound=0.5,
                  index_cols=[0,1,2],
                  use_cols=[3],
                  par_name_base='drnelev-cn',
                  pargp='delev-cn'
                 )

#### 5. Paramterise the K array

In [ ]:
# K pars
# different variogram for k
v = pyemu.geostats.ExpVario(contribution=1.0,a=400)
k_gs = pyemu.geostats.GeoStruct(variograms=v, transform='log')
# single layer
# grid based HK
_=pf.add_parameters("hk_layer_1.ref", # array like file
                  par_type='grid', 
                  lower_bound=0.01, 
                  upper_bound=100.,
                  par_name_base='hk-gr',
                  pargp='hk-gr',
                  geostruct=k_gs,
                  zone_array=ib[0]  # zone controls only active cells
                 )
# constant HK
_=pf.add_parameters("hk_layer_1.ref", 
                  par_type='constant', 
                  lower_bound=0.01, 
                  upper_bound=100.,
                  par_name_base='hk-cn',
                  pargp='hk-cn',
                  zone_array=ib[0]
                 )

#### 6. Add some obs

In [ ]:
# add some obs
obs_locs.to_csv(os.path.join(pf.new_d, "Cox_Operational_Targets.dat"))
# convert binary output to nice tabular list file that we can pass to pstfrom
hdobsnme, wlobsnme = utils.hds_bin2csv(d=pf.new_d,
                                       modelfname=f"{base_m.name}.nam")

In [ ]:
hdobsnme, wlobsnme

In [ ]:
pf.add_observations(
    wlobsnme, ## "welobs.csv"
    prefix="wl",
    index_cols=["sitename", 'k', 'i', 'j', 'kper'],
    use_cols='obsval',
    ofile_sep=','
)

#### 7. Let's add some more -- incl some to just track (predictions?!)

In [ ]:
# add less-than inequality as obs too (only for history model)
_=pf.add_observations(
    hdobsnme, ## "hds.csv"
    insfile="lthds.csv.ins",
    prefix="less_hd",
    index_cols=['k', 'i', 'j', 'kper'],
    use_cols='obsval',
    ofile_sep=',',
    obsgp="less_hd"
)

# Add this file as outputs to track
# can be as array files if we want
# pf.add_observations("hds_kper0_lay0.csv",  prefix="hd")
# or whole tabulated model output
_=pf.add_observations(
    hdobsnme, ## "hds.csv"
    prefix="hd",
    index_cols=['k', 'i', 'j', 'kper'],
    use_cols='obsval',
    ofile_sep=','
)

#### 8. We better make sure that that Post-Processing script is run at forward run time

In [ ]:
# need to add the above function to our forward run script so that
# it is also run at model run time
pf.add_py_function(
    os.path.join('..', 'scripts', "utils.py"),
    call_str=f"hds_bin2csv('.', modelfname='{base_m.name}.nam')",
    is_pre_cmd=False
)

#### 9. Ok -- lets finish this off!

In [ ]:
pst, pf = build.continue_setup_pst(pf, base_m, nreals=nreals, vis_cov=False)

### for fun let's build and view the prior paramter cov:

In [ ]:
cov = pf.build_prior().df()
 # just a trick to sort indexes so that closely located pars in space are close in the covariance matrix 
x = cov.values.copy()
x[x==0.0] = np.NaN

In [ ]:
fig = plt.figure(figsize=(8, 8), constrained_layout=True)
gs = plt.GridSpec(1, 2, width_ratios=[1, 0.01], wspace=0.05, figure=fig)
ax = plt.subplot(gs[:, 0])
im = ax.imshow(x, interpolation='none')
ax.set_facecolor('k')

gp = pst.parameter_data.loc[cov.index,:].groupby('pargp')
mapper = pd.Series(dict([[v.min(),k] for k,v in gp.indices.items()]))

_ = ax.yaxis.set_ticks(mapper.index, mapper.values, rotation=30, fontsize=12)
_ = ax.xaxis.set_ticks(mapper.index, mapper.values, rotation=90, fontsize=12)
del cov, x

# set-up PEST interface for projection model

In [ ]:
pred_pst, _ = build.setup_pst("Dunedin_Pred_base", nreals=nreals)

_________
# Prior MC
#### using `pestpp-ies`, settting `noptmax=-1` here we run the history period prior Monte Carlo analysis
* using the number of realizations specified by `nreals`
* will run in parallel locally using the number of cores specified by `num_workers` in `../scripts/build.py`
* creates a new directory called `master_hist_prior/` that will contain the PEST++ output from the parallel Monte Carlo
* upon running will generate worker directories

In [ ]:
# either use base_m model object as defined above or explicitly pass str:
m_d = "master_hist_prior"
noptmax=-1
pst = "Dunedin_SS_base.pst"
nworkers = 24
utils.prep_and_run(pst, t_d="template_hist_ss", m_d=m_d, nreals=nreals, 
                   noptmax=noptmax, nworker=nworkers)

## Load prior simulated output ensemble

In [ ]:
# read modflow model
m_d = "master_hist_prior"
m_d =os.path.join("..", m_d)
m = flopy.modflow.Modflow.load(
    f="SouthDun_SS.nam", 
    model_ws=m_d, 
    version='mfnwt',
    exe_name='mfnwt.exe', 
    verbose=False, 
    check=True
)
# PROCESS SWEEP OUTPUTS From Prior MC
# re-read pst control file from master dir
try:
    pst.filename = Path(pst.filename)
    pst = pyemu.Pst(os.path.join(m_d, pst.filename.name))
except (AttributeError, NameError):
    pst = "Dunedin_SS_base.pst"
    pst = pyemu.Pst(os.path.join(m_d, pst))
# get pest obs data from pest control file object
obs = pst.observation_data

# pulling pest file name from pst object
pstfnme = Path(pst.filename)
# show initial phi componenets
print("Group phi components from base model run")
display(pd.DataFrame.from_dict(pst.phi_components, orient='index'))
# re ensemble outputs
oe_pr = utils.try_load_ensemble(pst, os.path.join(m_d, f"{pstfnme.stem}.0.obs.csv"), kind='obs')
# read obs + noise ensemble (IES drew this from obs weights when we ran prior)
# revist weight on water level obs
nnzobs = obs.loc[pst.nnz_obs_names]
wlobs = nnzobs.loc[nnzobs.oname=="wl"].index

### Let's look at mapping prior probability of inundation

In [ ]:
hdobs = obs.loc[obs.oname == "hd"].astype({c: int for c in ['kper', 'k', 'i', 'j']})
hdobs['top'] = m.dis.top.array[tuple(hdobs[['i', 'j']].T.values)]
hdvtop = oe_pr.T.loc[hdobs.index].sub(hdobs.top, axis=0)
# calc probabilities of exceed for every output
# Transpose obs ensemble (ensemble outputs), slice for just hd obs,
# substract model top from every realisation,
# where positive simulated head exceeds model top, count reals where positive, divide by nreal
hdobs['prob'] = (hdvtop > 0).sum(axis=1) / oe_pr.shape[0]

# create an array from these obs -- WILL NEED TO BE DIF IF MULTPLE KPER AND LAYERS
ar_prior = np.zeros((m.nrow, m.ncol))  # blank array

# add elements from dataframe
ar_prior[tuple(hdobs[['i', 'j']].T.values)] = hdobs.prob

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
im = ax.imshow(np.ma.masked_where(m.bas6.ibound.array[0] == 0, ar_prior * 100),
               cmap="plasma", interpolation='none')
ax.set_title('Prior probability of groundwater inundation', fontsize='12')
fig.colorbar(im)
fig.tight_layout()

### Prior total drain flux sim out -versus- the estimated observation + error

In [ ]:
# Plotting ensemble output histograms
#load obs+noise ensemble
obsplus = utils.try_load_ensemble(pst, os.path.join(m_d, f"{pstfnme.stem}.obs+noise.csv"), kind='obs')
# just slice out drain obs
drnsumobs = obs.loc[obs.oname=="drnsum"]#.astype({c:int for c in ['kper','k','i','j']})
# Prior sim out of drain sum ob
drnsumoe_pr = oe_pr.loc[:, drnsumobs.index]
# obs plus noise for output 
dnobsplus = obsplus.loc[:, drnsumobs.index]
# plot histos
fig, ax = plt.subplots(1,1, figsize=(8,6))
_ = drnsumoe_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
_ = dnobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
ax.set_title(label='Storm / wastewater flux (prior -vs- obs)', fontsize=7)
ax.tick_params(axis='x', labelsize=7)
ax.tick_params(axis='y', labelsize=7)
ax.set_xlabel('Flux m$^3$/day', fontsize=7)
ax.set_ylabel('Num. reals', fontsize=7)

### Prior groundwater level sim out -versus- all water level observations + the estimated error

In [ ]:
# Prior simulatout outputs for WL obs vs obs+noise reals
w_obs = obs.loc[obs.index.str.contains('sitename') & (obs.weight>0)].astype({'i':int, 'j':int})

for ob in w_obs.index:
    top = m.dis.top.array[(w_obs.loc[ob].i, w_obs.loc[ob].j)]
    # Prior sim out
    tp_pr = oe_pr.loc[:, ob]
    # OBS 
    # obsplus = pd.read_csv(os.path.join(m_d, "Dunedin_SS_base.obs+noise.csv"), index_col=0)
    tpobsplus = obsplus.loc[:, ob]
    
    fig, axes = plt.subplots(1,2, figsize=(10,5))
    ax = axes[0]
    ax2 = axes[1]
    ax2.imshow(np.ma.masked_where(m.bas6.ibound.array[0] == 0, ar_prior * 100),
               cmap="plasma", interpolation='none')
    ax2.scatter(*w_obs.loc[ob, ['j', 'i']].values, marker='v', color='r')
    ax2.scatter(*w_obs.loc[ob, ['j', 'i']].values, marker='o', color='w', fc='none', s=120, lw=2)
    tp_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
    tpobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
    ys = ax.get_ylim()
    ax.plot((top, top), ys, color='k', ls='--')
    ax.set_title(ob, fontsize=6)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    ax.set_xlabel('Groundwater level m (OMD)', fontsize=7)
    ax.set_ylabel('Num. reals', fontsize=7)

### Refine weights 
#### de-weight groundwater level observations located in e.g., the perched dune aquifer system (before history matching)

In [ ]:
# first increase variance on highly uncertain drain obs estimate
obs.loc[drnsumobs.index, 'weight'] = 1/500
# drop moana rua shallow
mrs = w_obs.loc[w_obs.sitename=='moana-rua-shallow'] 
obs.loc[mrs.index, 'weight'] = 0
# drop Tonga Park deep
tpd = w_obs.loc[w_obs.sitename=='tonga-park-deep'] 
obs.loc[tpd.index, 'weight'] = 0
# reweight fitzroy street
ftz = w_obs.loc[w_obs.sitename=='fitzroy-st'] 
obs.loc[ftz.index, 'weight'] = 0
# reweight Turukina Rd
trd = w_obs.loc[w_obs.sitename=='turakina-rd'] 
obs.loc[trd.index, 'weight'] = 0
# reweight Scout hall shallow
shs = w_obs.loc[w_obs.sitename=='scout-hall-shallow'] 
obs.loc[shs.index, 'weight'] = 0
# reweight OMES
oms = w_obs.loc[w_obs.sitename=='omes'] 
obs.loc[oms.index, 'weight'] = 0
# reweight holiday-park
hyd = w_obs.loc[w_obs.sitename=='holiday-park'] 
obs.loc[hyd.index, 'weight'] = 0
# reweight Richardson
ric = w_obs.loc[w_obs.sitename=='richardson-st'] 
obs.loc[ric.index, 'weight'] = 0
# reweight ravelston
rav = w_obs.loc[w_obs.sitename=='ravelston'] 
obs.loc[rav.index, 'weight'] = 0
# reweight forbury
fby = w_obs.loc[w_obs.sitename=='forbury-park-raceway'] 
obs.loc[fby.index, 'weight'] = 0

#### Draw realisations of observation plus noise (based on observation weights)

In [ ]:
# redraw obs plus noise
obsplus = pyemu.ObservationEnsemble.from_gaussian_draw(
    pst, num_reals=nreals, fill=True)
obsplus.add_base()
obsplus = obsplus._df
# force drawn obs reals for drain to be < -500
dnobsplus = obsplus.loc[:, drnsumobs.index]
dnobsplus[dnobsplus > 0] = 0
obsplus.loc[:, drnsumobs.index] = dnobsplus
# for fun plot drain prior with obs+noise reals
fig, ax = plt.subplots(1,1, figsize=(6,4))
_ = drnsumoe_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
_ = obsplus.loc[:, drnsumobs.index].hist(
    ax=ax, bins=25, color='r', alpha=0.5, density=False)
ax.set_title(label='Storm / wastewater flux (prior -vs- obs)', fontsize=7)
ax.tick_params(axis='x', labelsize=7)
ax.tick_params(axis='y', labelsize=7)
ax.set_xlabel('Flux m$^3$/day', fontsize=7)
ax.set_ylabel('Num. reals', fontsize=7)

# similarly clip drawn wl noise obs to below model surface
w_obs['top'] = m.dis.top.array[tuple(w_obs[['i','j']].values.T)]
wlobsplus = obsplus.loc[:, w_obs.index]
wlobsplus = wlobsplus.where(~wlobsplus.gt(w_obs.top), w_obs.top, axis=1)
obsplus.loc[:, w_obs.index] = wlobsplus
# for ob in w_obs.index:
#     top = m.dis.top.array[(w_obs.loc[ob].i, w_obs.loc[ob].j)]
#     # Prior sim out
#     tp_pr = oe_pr.loc[:, ob]
#     # OBS 
#     # obsplus = pd.read_csv(os.path.join(m_d, "Dunedin_SS_base.obs+noise.csv"), index_col=0)
#     tpobsplus = obsplus.loc[:, ob]
    
#     fig, axes = plt.subplots(1,2, figsize=(10,5))
#     ax = axes[0]
#     ax2 = axes[1]
#     ax2.imshow(np.ma.masked_where(m.bas6.ibound.array[0] == 0, ar_prior * 100),
#                cmap="plasma", interpolation='none')
#     ax2.scatter(*w_obs.loc[ob, ['j', 'i']].values, marker='v', color='r')
#     ax2.scatter(*w_obs.loc[ob, ['j', 'i']].values, marker='o', color='w', fc='none', s=120, lw=2)
#     tp_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
#     tpobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
#     ys = ax.get_ylim()
#     ax.plot((top, top), ys, color='k', ls='--')
#     ax.set_title(ob, fontsize=6)
#     ax.tick_params(axis='x', labelsize=7)
#     ax.tick_params(axis='y', labelsize=7)
#     ax.set_xlabel('Groundwater level m (OMD)', fontsize=7)
#     ax.set_ylabel('Num. reals', fontsize=7)

### Balance objective function -- focus on water level obs

In [ ]:
# pdc work
nnzobs = obs.loc[pst.nnz_obs_names]
# set less_than obsplus to be equal to obs val (as we are determining them to be model top always)
lessthanobs = nnzobs.loc[nnzobs.obgnme.str.startswith('less_')]
# modify obs+noise to remove noise from less than obs
obsplus.loc[:, lessthanobs.index] = lessthanobs.obsval.values
#display(obsplus.loc[:, lessthanobs.index].head())
# write modified obs+noise ensemble to prior master and template
# pass back to an obsensemble object
obsplus = pyemu.ObservationEnsemble(pst, obsplus)
obsplus.to_binary(os.path.join(m_d, "Dunedin_SS_base_rw.obs+noise.jcb"))
obsplus.to_binary(os.path.join("..", "template_hist_ss", "Dunedin_SS_base_rw.obs+noise.jcb"))

# min mean and max of simulated outputs (used to determine conflicts)
nnzobs['minout'] = oe_pr.loc[:, nnzobs.index].min()
nnzobs['meanout'] = oe_pr.loc[:, nnzobs.index].mean()
nnzobs['maxout'] = oe_pr.loc[:, nnzobs.index].max()

# min mean and max of obs+noise reals (used to determine conflicts)
nnzobs['obsmin'] = obsplus.loc[:, nnzobs.index].min()
nnzobs['obsmean'] = obsplus.loc[:, nnzobs.index].mean()
nnzobs['obsmax'] = obsplus.loc[:, nnzobs.index].max()

# need to treat less than obs and normal obs a bit different
lessthanobs = nnzobs.loc[lessthanobs.index] # only conflict where simmin>obsmax
normalobs = nnzobs.loc[nnzobs.index.difference(lessthanobs.index)] # also conflict where simmax<obsmin

# pull out conflicting obs
conflict_obs = pd.concat(
    [normalobs.loc[normalobs.minout > normalobs.obsmax,:],
     normalobs.loc[normalobs.maxout < normalobs.obsmin,:],
     lessthanobs.loc[lessthanobs.minout > lessthanobs.obsmax,:]]
)
display(conflict_obs)
#display(conflict_obs.groupby('obgnme').count())
#display(obs.loc[conflict_obs.index, ['weight']])

# set weight of conflicting obs to zero
obs.loc[conflict_obs.index, 'weight'] = 0
#display(obs.loc[conflict_obs.index, ['weight']])
pst.observation_data = obs
obsplus=obsplus._df

In [ ]:
# Balancing weights
# Get mean residual
# first make sure obplus and oe are alignable
obsplus.index = obsplus.index.astype(str)

# actual residual
res = oe_pr.sub(obsplus)
# reset gt and lt obs that satisfy inequality
ltobs = obs.loc[obs.obgnme.str.startswith('less_')]
# ltobs where res is < 0
res.loc[:, ltobs.index] = res.loc[:, ltobs.index].clip(0, None)
obs['swr'] = obs.weight * res.mean()**2
# initial phi contribs for groups
phi_comps = obs.loc[pst.nnz_obs_names].groupby('obgnme')[['swr']].sum()

maxgpcont = phi_comps.max()['swr']
phi_comps['desired'] = maxgpcont
# reduce relative on inequality and drain
phi_comps.loc[phi_comps.index == "less_hd", 'desired'] *= 3.
phi_comps.loc[phi_comps.index == "oname:drnsum_otype:lst_usecol:sum", 'desired'] *= 1.0e-1
display(phi_comps)

# calculate weight multiplier for each group
wgtmul = (phi_comps.desired/phi_comps.swr)
wgtmul

In [ ]:
# apply mult to obs weights
obs.loc[pst.nnz_obs_names, 'weight'] *= \
    wgtmul.loc[obs.loc[pst.nnz_obs_names, 'obgnme']].values

#### Prep for history matching

In [ ]:
pst.pestpp_options["ies_obs_en"] = "Dunedin_SS_base_rw.obs+noise.jcb"
pst.pestpp_options["ies_restart_observation_ensemble"] = "Dunedin_SS_base_rw.0.obs.jcb"
pst.pestpp_options["ies_par_en"] = "Dunedin_SS_base.0.par.jcb"
pst.pestpp_options["ies_subset_size"] = int(0.1 * nreals)
# 10% of nreals (later version of PESTPP can just use -10)
# save update pst control file (in master prior for now)
if os.path.exists(
        os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst.bckup")
):
    shutil.copy(os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst.bckup"),
                os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst"))
else:
    shutil.copy(os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst"),
                os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst.bckup"))
pst.write(os.path.join(m_d, "Dunedin_SS_base_rw.pst"), version=2)
pst.write(os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst"), version=2)
# copy files that we need in template
shutil.copy(os.path.join(m_d, "Dunedin_SS_base.0.obs.jcb"),
            os.path.join("..", "template_hist_ss", "Dunedin_SS_base_rw.0.obs.jcb"))
shutil.copy(os.path.join(m_d, "Dunedin_SS_base.0.par.jcb"),
            os.path.join("..", "template_hist_ss", "Dunedin_SS_base.0.par.jcb"))

___________________
# History matching
#### using `pestpp-ies`, settting `noptmax` here we history match to gw levels obs, total drain flux and inequality constraints
* using the number of realizations specified by `nreals`
* will run in parallel locally using the number of cores specified by `num_workers` in `../scripts/build.py`
* creates a new directory called `master/` that will contain the PEST++ output from each iteration of history matching
* upon running will generate worker directories

In [ ]:
# either use base_m model object as defined above or explicitly pass str:
noptmax=32 # number of pestpp-ies iterations
num_workers = 50 # number of parallel runs
pstfile = "Dunedin_SS_base.pst" # pst control file
utils.prep_and_run(pstfile, t_d="template_hist_ss", 
                   nreals=nreals, noptmax=noptmax, nworker=num_workers)

In [ ]:
phiprog_act = pd.read_csv(Path(m_d, "Dunedin_ss_base.phi.actual.csv"), index_col=0)
phiprog_meas = pd.read_csv(Path(m_d, "Dunedin_ss_base.phi.meas.csv"), index_col=0)
fig, ax = plt.subplots(1, figsize=(8,6))
# phiprog_act.drop(['total_runs', 'mean', 'standard_deviation', 'min', 'max'], axis=1).T.transform(np.log10).boxplot(ax=ax)
phiprog_meas.drop(['total_runs', 'mean', 'standard_deviation', 'min', 'max'], axis=1).T.transform(np.log10).boxplot(ax=ax)

# Process the posterior for history period
#### Let's plot the posterior probability of groundwater inundation

In [ ]:
# new master directory
m_d = os.path.join("..", "master")

# pestpp-ies iteration
it = 14 # noptmax

# load posterior ensemble
oe_po = utils.try_load_ensemble(pst, os.path.join(m_d, f"Dunedin_SS_base.{it}.obs.jcb"), 'obs')

# get pest obs data
obs = pst.observation_data

# just hd outputs
hdobs = obs.loc[obs.obgnme == "oname:hd_otype:lst_usecol:obsval"].astype({c:int for c in ['kper','k','i','j']})

# add column that aligns model top info to hd output names
hdobs['top'] = m.dis.top.array[tuple(hdobs[['i','j']].T.values)]

# calc probabilities of exceed for every output
# Transpose obs ensemble (ensemble outputs), slice for just hd obs, 
# substract model top from every realisation,
# where positive simulated head exceeds model top, count reals where positive, divide by nreal
hdobs['prob'] = (oe_po.T.loc[hdobs.index].sub(hdobs.top, axis=0) > 0).sum(axis=1)/oe_po.shape[0]

# create an array from these obs -- WILL NEED TO BE DIF IF MULTPLE KPER AND LAYERS
ar_post = np.zeros((m.nrow, m.ncol)) # blank array

# add elements from dataframe
ar_post[tuple(hdobs[['i','j']].T.values)] = hdobs.prob

fig = plt.figure(figsize=(8,4))
pgrid = plt.GridSpec(1, 3, width_ratios=[1,1,0.1], figure=fig)
ax=plt.subplot(pgrid[0])
ax2=plt.subplot(pgrid[1])
cax=plt.subplot(pgrid[2])
im = ax.imshow(np.ma.masked_where(m.bas6.ibound.array[0] == 0, ar_prior),
               cmap="plasma", interpolation='none')
ax.set_title('Prior P(GW_inundation)', fontsize='12')
im2 = ax2.imshow(np.ma.masked_where(m.bas6.ibound.array[0] == 0, ar_post),
               cmap="plasma", interpolation='none')
ax2.set_title('Post P(GW_inundation)', fontsize='12')
fig.colorbar(im2, cax=cax)
plt.show()

#### We plot the prior versus posterior drain flux distributions

In [ ]:
drnsumobs = obs.loc[obs.index.str.startswith('oname:drnsum')]#.astype({c:int for c in ['kper','k','i','j']})
# Prior sim out
drnsumoe_pr = oe_pr.loc[:, drnsumobs.index]
# Posterior sim out
drnsumoe_po = oe_po.loc[:, drnsumobs.index]

# OBS
obsplus = pyemu.ObservationEnsemble.from_binary(
    pst, os.path.join(m_d, "Dunedin_SS_base_rw.obs+noise.jcb"))
# obsplus = pd.read_csv(os.path.join(m_d, "Dunedin_SS_base.obs+noise.csv"), 
#                       index_col=0)
dnobsplus = obsplus.loc[:, drnsumobs.index]

fig, ax = plt.subplots(1,1, figsize=(6,4))
drnsumoe_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
drnsumoe_po.hist(ax=ax, bins=25, color='b', alpha=0.5, density=False)
dnobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
ax.set_title(label='Storm / wastewater flux (prior -vs- post. -vs- obs)', fontsize=7)
ax.tick_params(axis='x', labelsize=7)
ax.tick_params(axis='y', labelsize=7)
ax.set_xlabel('Flux m$^3$/day', fontsize=7)
ax.set_ylabel('Num. reals', fontsize=7)
ax.set_xlim(-6000, 1000)
#ax.set_ylim(0, 50)
#plt.savefig("plots/prior_drn_flux")

#### Prior versus posterior ensemble outputs for the gw level observation locations

In [ ]:
w_obs = obs.loc[
    obs.index.str.contains('sitename') & (obs.kper=='0')
].astype({'i':int, 'j':int})

m.dis.top.array[(w_obs.loc[ob].i, w_obs.loc[ob].j)]

for ob in w_obs.index:
    top = m.dis.top.array[(w_obs.loc[ob].i, w_obs.loc[ob].j)]
    # Prior sim out
    tp_pr = oe_pr.loc[:, ob]
    # Posterior sim out
    tp_po = oe_po.loc[:, ob]
    # OBS 
    # obsplus = pd.read_csv(os.path.join(m_d, "Dunedin_SS_base.obs+noise.csv"), index_col=0)
    tpobsplus = obsplus.loc[:, ob]
    
    fig, axes = plt.subplots(1,2, figsize=(10,5))
    ax = axes[0]
    ax2 = axes[1]
    ax2.imshow(np.ma.masked_where(m.bas6.ibound.array[0] == 0, ar_post * 100),
               cmap="plasma", interpolation='none')
    ax2.scatter(*w_obs.loc[ob, ['j', 'i']].values, marker='v', color='r')
    ax2.scatter(*w_obs.loc[ob, ['j', 'i']].values, marker='o', color='w', fc='none', s=120, lw=2)
    tp_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
    tp_po.hist(ax=ax, bins=25, color='b', alpha=0.5, density=False)
    tpobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
    ys = ax.get_ylim()
    ax.plot((top, top), ys, c='k', ls='--')
    ax.plot((w_obs.loc[ob, 'obsval'], w_obs.loc[ob, 'obsval']), ys, c='r', ls='--')
    ax.set_title(ob, fontsize=8)
    #ax.set_title(label='Storm / wastewater flux (prior -vs- obs)', fontsize=7)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    #ax.set_xlim(98, 103)
    ax.set_xlabel('groundwater level m (OMD)', fontsize=7)
    ax.set_ylabel('Num. reals', fontsize=7)

#### % reduction in uncertainty for the groundwater level prediction (for the history matching period)

In [ ]:
# re ensemble outputs
#load obs po ensemble using Brioch's try_load_ensemble function
obs_ens_po = utils.try_load_ensemble(pst, os.path.join(m_d, f"Dunedin_SS_base.{it}.obs.jcb"), kind='obs')
obs_reals_po = obs_ens_po.T
#obs_reals
obs_po = obs_reals_po.loc[obs_reals_po.index.str.contains('oname:hd_otype')] #.astype({c:int for c in ['i','j']})

obs_ens_pr = utils.try_load_ensemble(pst, os.path.join(m_d, "Dunedin_SS_base.0.obs.jcb"), kind='obs')
obs_reals_pr= obs_ens_pr.T
#obs_reals
obs_pr = obs_reals_pr.loc[obs_reals_pr.index.str.contains('oname:hd_otype')] #.astype({c:int for c in ['i','j']})
percent = 100 * (1.0 - obs_po.std(axis=1)/obs_pr.std(axis=1))
ar = np.zeros(m.dis.top.shape) * np.nan
ar[tuple(
    pst.observation_data.loc[obs_pr.index][['i', 'j']].astype(int).values.T
)] =  percent # par_obs.loc[:, 0].values

plt.figure()
plt.imshow(np.ma.masked_where(ar<0, ar), cmap = 'plasma', vmin=0, vmax=100)
plt.colorbar()